SUMMARY: Export data from excel to sqlite db, and then query duplicate Sender ID and Beneficiary ID. 
After that, save the result back to excel.

In [1]:
import pandas as pd
import sqlite3 as sq

from sqlalchemy import create_engine

In [3]:
# read excel, save to dataframe
df = pd.read_excel('mtpg_report.xlsx',header=0, index_col=0)

In [4]:
df.shape

(22907, 4)

In [5]:
df.head()

,Transaction Type,Merchant Name,Sender ID,Beneficiary ID
Invoice Number,,,,
CK414C0KC,Transfer IN,WAL,F01G00******7F23,A878Z2A64Z
I0YK90Y4LMLMY5CLK0,Transfer OUT,MCP,00220371F1,A57ZZZ7A444Z6
I0YLMK10M4KKYC5C5C,Transfer OUT,PER,00220371F1,C8Z8952462ACA9
I0Y5KYY191K1Y9C4K0,Transfer OUT,BAN,00220371F1,Z985AZ2445
I0Y4M0L94M1LK9150,Transfer OUT,BAR,003017G714,5CZ6ZAZCZ6A25CC


In [6]:
# preparing connection to db
engine = create_engine('sqlite:///mtpg_report.db', echo=True)
sqlite_connection = engine.connect()

2020-09-26 17:13:56,638 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-26 17:13:56,639 INFO sqlalchemy.engine.base.Engine ()
2020-09-26 17:13:56,641 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-26 17:13:56,641 INFO sqlalchemy.engine.base.Engine ()


In [8]:
# export df to sqlite db, if exists then replace
sqlite_table = "mtpg"
df.to_sql(sqlite_table, sqlite_connection, if_exists='replace')


2020-09-26 17:30:53,608 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("mtpg")
2020-09-26 17:30:53,610 INFO sqlalchemy.engine.base.Engine ()
2020-09-26 17:30:53,612 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("mtpg")
2020-09-26 17:30:53,614 INFO sqlalchemy.engine.base.Engine ()
2020-09-26 17:30:53,617 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-26 17:30:53,619 INFO sqlalchemy.engine.base.Engine ()
2020-09-26 17:30:53,625 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("mtpg")
2020-09-26 17:30:53,627 INFO sqlalchemy.engine.base.Engine ()
2020-09-26 17:30:53,635 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'mtpg' AND type = 'table'
2020-09-26 17:30:53,638 INFO sqlalchemy.engine.base.Engine ()
2020-09-26 17:30:53,643 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("mtpg")
2020-09-26 

In [9]:
sqlite_connection.close()

In [10]:
# check creted db by selecting 5 records
conn = sq.connect("mtpg_report.db")
df_mtpg = pd.read_sql_query ("select * from mtpg limit 5;", conn)
df_mtpg

,Invoice Number,Transaction Type,Merchant Name,Sender ID,Beneficiary ID
0,CK414C0KC,Transfer IN,WAL,F01G00******7F23,A878Z2A64Z
1,I0YK90Y4LMLMY5CLK0,Transfer OUT,MCP,00220371F1,A57ZZZ7A444Z6
2,I0YLMK10M4KKYC5C5C,Transfer OUT,PER,00220371F1,C8Z8952462ACA9
3,I0Y5KYY191K1Y9C4K0,Transfer OUT,BAN,00220371F1,Z985AZ2445
4,I0Y4M0L94M1LK9150,Transfer OUT,BAR,003017G714,5CZ6ZAZCZ6A25CC


In [11]:
#  sender id duplicated
str_query = "SELECT * \
            FROM mtpg \
                WHERE (\"Sender ID\") IN \
                (SELECT \"Sender ID\" \
                    FROM mtpg WHERE \"Merchant Name\" = \"WAL\" \
                    AND \"Transaction Type\" = \"Transfer OUT\" \
                    GROUP BY \"Sender ID\" \
                    HAVING COUNT (*)>1 \
                    )\
            ORDER BY \"Sender ID\" DESC"
df_mtpg = pd.read_sql_query (str_query, conn)
df_mtpg

,Invoice Number,Transaction Type,Merchant Name,Sender ID,Beneficiary ID
0,1M955C0K,Transfer OUT,WAL,1GGG772224,A95ACA9665
1,1M9590LC,Transfer OUT,WAL,1GGG772224,C8Z8CA257C4A44
2,1M9590LC,Transfer OUT,WAL,1GGG772224,C8Z8CA257C4A44
3,1M9C01MY,Transfer OUT,WAL,1GGG772224,8A95228856
4,1M9C01MY,Transfer OUT,WAL,1GGG772224,8A95228856
...,...,...,...,...,...
6098,1M9CCYC5,Transfer OUT,WAL,10022FF032,825ZCC27A9
6099,1M9L1LMC,Transfer OUT,WAL,10020GG84G,A52ZZA84A664Z
6100,1M9L1LMC,Transfer OUT,WAL,10020GG84G,A52ZZA84A664Z
6101,1M9L1LKC,Transfer OUT,WAL,1001525F10,797ZCZ658A


In [12]:
# beneficiary id, dan sender id duplicated
str_query = "SELECT * \
            FROM mtpg \
                WHERE (\"Beneficiary ID\") IN \
                (SELECT \"Beneficiary ID\" \
                 FROM mtpg WHERE \"Sender ID\" IN  \
                    (SELECT \"Sender ID\" \
                        FROM mtpg WHERE \"Merchant Name\" = \"WAL\" \
                        AND \"Transaction Type\" = \"Transfer OUT\" \
                        GROUP BY \"Sender ID\" \
                        HAVING COUNT (*)>1 \
                    )\
                GROUP BY \"Beneficiary ID\" \
                HAVING COUNT (*)>1 \
                )\
            ORDER BY \"Sender ID\" DESC"
df_mtpg = pd.read_sql_query (str_query, conn)
df_mtpg

,Invoice Number,Transaction Type,Merchant Name,Sender ID,Beneficiary ID
0,1M9590LC,Transfer OUT,WAL,1GGG772224,C8Z8CA257C4A44
1,1M9590LC,Transfer OUT,WAL,1GGG772224,C8Z8CA257C4A44
2,1M9C01MY,Transfer OUT,WAL,1GGG772224,8A95228856
3,1M9C01MY,Transfer OUT,WAL,1GGG772224,8A95228856
4,1M9C0454,Transfer OUT,WAL,1GGG772224,8A95228856
...,...,...,...,...,...
5578,1M9CCYC5,Transfer OUT,WAL,10022FF032,825ZCC27A9
5579,1M9L1LMC,Transfer OUT,WAL,10020GG84G,A52ZZA84A664Z
5580,1M9L1LMC,Transfer OUT,WAL,10020GG84G,A52ZZA84A664Z
5581,1M9L1LKC,Transfer OUT,WAL,1001525F10,797ZCZ658A


In [13]:
# invoice no, beneficiary id, dan sender id duplicated

str_query = "SELECT * \
            FROM mtpg \
                WHERE (\"Invoice Number\") IN \
                (SELECT \"Invoice Number\" \
                    FROM mtpg WHERE \"Beneficiary ID\" IN \
                    (SELECT \"Beneficiary ID\" \
                     FROM mtpg WHERE \"Sender ID\" IN  \
                        (SELECT \"Sender ID\" \
                            FROM mtpg WHERE \"Merchant Name\" = \"WAL\" \
                            AND \"Transaction Type\" = \"Transfer OUT\" \
                            GROUP BY \"Sender ID\" \
                            HAVING COUNT (*)>1 \
                        )\
                    GROUP BY \"Beneficiary ID\" \
                    HAVING COUNT (*)>1 \
                    )\
                GROUP BY \"Invoice Number\" \
                HAVING COUNT (*)>1) \
                ORDER BY \"Sender ID\" DESC"
df_mtpg = pd.read_sql_query (str_query, conn)
df_mtpg

,Invoice Number,Transaction Type,Merchant Name,Sender ID,Beneficiary ID
0,1M9590LC,Transfer OUT,WAL,1GGG772224,C8Z8CA257C4A44
1,1M9590LC,Transfer OUT,WAL,1GGG772224,C8Z8CA257C4A44
2,1M9C01MY,Transfer OUT,WAL,1GGG772224,8A95228856
3,1M9C01MY,Transfer OUT,WAL,1GGG772224,8A95228856
4,1M9C0454,Transfer OUT,WAL,1GGG772224,8A95228856
...,...,...,...,...,...
2985,1M9CCYC5,Transfer OUT,WAL,10022FF032,825ZCC27A9
2986,1M9L1LMC,Transfer OUT,WAL,10020GG84G,A52ZZA84A664Z
2987,1M9L1LMC,Transfer OUT,WAL,10020GG84G,A52ZZA84A664Z
2988,1M9L1LKC,Transfer OUT,WAL,1001525F10,797ZCZ658A


In [ ]:
# save result to an excel file
df_mtpg.to_excel("mtpg_filter_duplicated.xlsx")